In [12]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw
from pathlib import Path
from scipy.ndimage import label
import pandas as pd
import h5py

In [4]:
def average_attentions(attention_list):    
    summed_att=np.zeros_like(np.load(attention_list[0]))
    count_non_zero= np.zeros_like(summed_att)
    for attention in attention_list:
        att=np.load(attention)
        att=np.nan_to_num(att)
        summed_att=summed_att+att
        count_non_zero=count_non_zero+(att>0).astype(np.uint32)
    return summed_att,summed_att/count_non_zero,count_non_zero

def get_image(image_name,slide_folder):
    slide_name,scene_nr=get_slide_name(image_name)
    slide=Image.open(slide_name)
    scene = slide.get_scene(scene_nr)
    wsi = wsi[..., ::-1]
    orig_size=np.uint32(np.array(size)/8)
    return wsi,orig_size


In [5]:


def get_high_attention_patch(attention_map,slide_folder,image_name,save_path):

    return 


def overlay_colormap_on_rgb(rgb_image, heatmap_image, image_name,save_path,high_att_coord, cmap='viridis', alpha=0.4):
    # Apply colormap to heatmap_image
    alpha_channel = np.ones((rgb_image.shape[0], rgb_image.shape[1]), dtype=np.uint8) * 255
    
    # Convert RGB image to RGBA format
    rgba_rgb_image = np.dstack((rgb_image, alpha_channel))
    cmap = plt.get_cmap(cmap)
    #inverted_cmap = cmap.reversed()
    heatmap_colored = (cmap(heatmap_image) * 255).astype(np.uint8)
    # Rotate heatmap by 90 degrees
    rotated_heatmap_coloured = np.fliplr(np.rot90(heatmap_colored, k=3))

    # Step 1: Expand the dimensions of resized_binary_mask
    resized_binary_mask = np.rot90(heatmap_image, k=3)

    # Resize rotated_heatmap to match the size of rgb_image
    resized_heatmap = Image.fromarray(rotated_heatmap_coloured).resize((rgb_image.shape[1],rgb_image.shape[0]), Image.BILINEAR)
    resized_binary_mask=Image.fromarray(resized_binary_mask).resize((rgb_image.shape[1],rgb_image.shape[0]), Image.BILINEAR)
    #resized_heatmap.save(image_name.replace(".npy",""))

    # Convert RGB image to PIL format
    pil_rgb_image = Image.fromarray(rgba_rgb_image)

    # Combine the two images with the specified alpha
    overlaid_image = Image.blend(pil_rgb_image, resized_heatmap, alpha)
    resized_binary_mask_expanded=np.expand_dims(np.flip(resized_binary_mask,1),-1)
    #overlaid_image.save("overlay_"+image_name.replace(".npy",""))
    overlaid_image=np.array(overlaid_image)
    zero_array = np.ones_like(overlaid_image)*255
    overlaid_image = np.where(resized_binary_mask_expanded, overlaid_image, zero_array)
    overlaid_image[:,:,3]=255
    #mask = np.all(overlaid_image[:, :, :3] == [153, 153, 153], axis=-1)

    #overlaid_image[mask]=[106, 113, 91,255]
    #overlaid_image=Image.fromarray(overlaid_image)
    overlaid_image=Image.fromarray(overlaid_image,mode='RGBA')
    draw = ImageDraw.Draw(overlaid_image)

    x,y=high_att_coord
    x,y=int(x/2),int(y/2)
    width = 64  # Replace with your width

    # Calculate coordinates of the right lower corner
    right_lower_corner = (y + width, x + width)

    # Draw a square
    draw.rectangle([y-width, x-width, right_lower_corner], outline="black",width=10)
    if save_path is not None:
        overlaid_image.save(Path(save_path)/image_name.replace(".npy",""))
    return pil_rgb_image


In [54]:

def map_attentions(image_id,attention_values,feature_dir="/mnt/ceph_vol/UCEC/features",feature_size=256,downscale=8):
    feature_size=int(feature_size/downscale)
    file= sorted(Path(feature_dir).glob(f"*{image_id}*.h5"))[0]
    all_ims=[]
    with h5py.File(file, 'r') as h5_file:
        # Assuming the data is stored in the root of the HDF5 file
        # Change 'dataset_name' to the actual name of your dataset # Adjust this if necessary
        coords,slide_sizes = h5_file["coords"],h5_file["slide_sizes"]
        # Return the first entry in the dataset
        all_coord_len=0
        scenes=list(np.unique([int(c[0]) for c in coords]))
        for i in scenes:
            filtered_coords=[c for c in list(coords) if c[0]==0]
            c_l=len(filtered_coords)
            im=np.zeros(slide_sizes[i])
            for coord,attention in zip(filtered_coords,attention_values[all_coord_len:all_coord_len+c_l]):
                x,y=int(coord[1]/downscale),int(coord[2]/downscale)
                im[x:x+feature_size,y:y+feature_size]=attention
            all_coord_len+=c_l   
        print("return im")       
        return im

In [56]:
done=[]
attention_folder="/mnt/ceph_vol/MAT/results/5foldcv/MTF_nll_surv_a0.0_lr2e-05_5foldcv_gc15_concat/tcga_ucec_MTF_nll_surv_a0.0_lr2e-05_5foldcv_gc15_concat_s1/bottom_attention"
save_folder="/mnt/ceph_vol/figure_vis_6/"
#extraction_list=["21-083_K1"]
attention_files=Path(attention_folder).glob("*.npy")
for attention_file in attention_files:
    image_id=attention_file.name.replace(".npy","")
    attention_values=np.load(attention_file)
    attention_mapped=map_attentions(image_id,attention_values)
    preview=get_preview(image_id)
    overlay_colormap_on_rgb(preview,attention_mapped,image_id,save_path,None)




return im


NameError: name 'get_preview' is not defined